

Tôi sẽ giúp bạn sửa lại controller để xử lý logic mà không cần dùng GPT. Đây là phiên bản đã được sửa đổi:

```javascript:backend/src/controllers/generateLearningFlexibleController.js
exports.generateFlexibleCard = async (req, res) => {
    try {
        console.log('Received request for flexible cards:', req.body);
        
        const { lessons } = req.body;
        const allResults = [];
        
        for (const lesson of lessons) {
            console.log('Processing flexible lesson:', lesson);
            
            const { question, structure, phrases } = lesson;
            const results = [];
            
            // Helper function to count characters
            const getUnderscores = (text) => '_'.repeat(text.length);
            
            // Helper function to replace part of sentence with underscores
            const replaceWithUnderscores = (sentence, phrase) => {
                return sentence.replace(phrase, getUnderscores(phrase));
            };
            
            // Helper function to hide everything except the phrase
            const hideExceptPhrase = (sentence, phrase) => {
                const words = sentence.split(' ');
                return words.map(word => 
                    phrase.includes(word) ? word : getUnderscores(word)
                ).join(' ');
            };

            // 1. Full sentence with no blanks
            const fullSentence = structure.replace('______', phrases[0]);
            results.push({
                description: "Hãy nghe mình nói và đọc lại",
                sentence_hide: fullSentence,
                sentence_en: fullSentence,
                sentence_vi: `Tôi là ${phrases[0]} của công ty ABC.`
            });

            // 2,3,4. Replace each phrase with underscores
            phrases.forEach(phrase => {
                const sentenceWithPhrase = structure.replace('______', phrase);
                results.push({
                    description: "Hãy dịch câu sau",
                    sentence_hide: replaceWithUnderscores(sentenceWithPhrase, phrase),
                    sentence_en: sentenceWithPhrase,
                    sentence_vi: `Tôi là ${phrase} của công ty ABC.`
                });
            });

            // 5,6,7. Only show the phrases
            phrases.forEach(phrase => {
                const sentenceWithPhrase = structure.replace('______', phrase);
                results.push({
                    description: "Hãy dịch câu sau",
                    sentence_hide: hideExceptPhrase(sentenceWithPhrase, phrase),
                    sentence_en: sentenceWithPhrase,
                    sentence_vi: `Tôi là ${phrase} của công ty ABC.`
                });
            });

            // 8. Full sentence with all words replaced by blanks
            results.push({
                description: "Hãy dịch câu sau",
                sentence_hide: getUnderscores(question),
                sentence_en: question,
                sentence_vi: "Bạn đang làm việc cho công ty nào vậy?"
            });

            allResults.push(...results);
        }
        
        console.log('Successfully generated flexible cards:', allResults.length);
        res.json(allResults);
        
    } catch (error) {
        console.error('Error in generateFlexibleCard:', {
            message: error.message,
            stack: error.stack
        });
        
        res.status(500).json({ 
            error: error.message,
            details: error.stack
        });
    }
};
```

### Những thay đổi chính:
1. Loại bỏ hoàn toàn phần gọi OpenAI API
2. Thêm các hàm helper để xử lý việc tạo dấu gạch dưới và thay thế text
3. Xử lý logic để tạo ra 8 kết quả theo yêu cầu:
   - Câu đầy đủ
   - 3 câu với từng phrase bị ẩn
   - 3 câu chỉ hiện phrase
   - Câu cuối với tất cả các từ bị ẩn

### Ưu điểm của cách này:
1. Nhanh hơn vì không cần gọi API
2. Không tốn chi phí API
3. Kết quả nhất quán và dễ kiểm soát
4. Không phụ thuộc vào OpenAI

### Lưu ý:
- Bạn có thể cần điều chỉnh logic tạo câu tiếng Việt để phù hợp với yêu cầu cụ thể
- Có thể thêm các validation để đảm bảo input đúng format
- Có thể tùy chỉnh độ dài của dấu gạch dưới theo yêu cầu